In [1]:
import sys
sys.path.append("..")

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

import numpy as np

# Trainer
from src.trainer.trainer import Trainer

# Networks
from src.models.mlp import MLP

## CIFAR10 dataset

In [2]:
root_path = "../data"
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor()
])
transform_test = transforms.ToTensor()

full_train = datasets.CIFAR10(root=root_path, train=True, download=True, transform=transform_train)

train_size = int(0.9 * len(full_train))
val_size = len(full_train) - train_size
train_set, val_set = random_split(full_train, [train_size, val_size])
test_set = datasets.CIFAR10(root=root_path, train=False, download=True, transform=transform_test)


batch_size = 128
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

In [3]:
img, label = full_train[0]
print(img.shape)

torch.Size([3, 32, 32])


## Table generation

In [5]:
# Init training variables
num_epochs = 10
num_iterations = 3
log_dir = "../logs"
accuracies = {
    "mlp": []
}

# ----- Init model paramerters -----

## MLP parameters:
input_size = 3 * 32 * 32
output_size = 10
hidden_sizes = [128, 64]

## ---------------------------------

for it in range(num_iterations):
    
    # ----- Models initialization -----
    mlp_model = MLP(
        input_size = input_size,
        hidden_sizes = hidden_sizes,
        output_size = output_size
    )

    # ---------------------------------

    # ----- Other parameter initialization -----
    mlp_criterion = nn.CrossEntropyLoss()
    mlp_optimizer = optim.SGD(mlp_model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    mlp_scheduler = optim.lr_scheduler.MultiStepLR(mlp_optimizer, milestones=[200, 250, 300], gamma=0.1)

    # ------------------------------------------

    # Generate lists
    models = {
        "mlp": mlp_model
    }
    criterions = [mlp_criterion]
    optimizers = [mlp_optimizer]
    schedulers = [mlp_scheduler]

    # -------------------------------------------
    
    trainer = Trainer(
        models = models,
        optimizers = optimizers,
        criterions = criterions,
        schedulers = schedulers,
        log_dir = log_dir
    )

    # Train models on dataset
    print("Training models...")
    trainer.train(
        num_epochs = num_epochs,
        train_loader = train_loader,
        val_loader = val_loader,
    )

    # Evaluate models on dataset
    print("Testing models...")
    test_accuracies = trainer.evaluate(test_loader = test_loader)

    for model_name, acc in test_accuracies.items():
        accuracies[model_name].append(acc)

Using device: cuda
Current GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Training models...
Training model : mlp


Training Epochs: 100%|██████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.41s/it, Train Acc=0.2237, Val Acc=0.2122, Train Loss=2.0284, Val Loss=2.0299]

Testing models...


Using device: cuda
Current GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Training models...
Training model : mlp


Training Epochs: 100%|██████████████████████████████████████████████████████████████████| 10/10 [00:23<00:00,  2.40s/it, Train Acc=0.2412, Val Acc=0.2496, Train Loss=1.9838, Val Loss=2.0426]

Testing models...


Using device: cuda
Current GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Training models...
Training model : mlp


Training Epochs: 100%|██████████████████████████████████████████████████████████████████| 10/10 [00:23<00:00,  2.40s/it, Train Acc=0.2408, Val Acc=0.2400, Train Loss=1.9978, Val Loss=2.0099]

Testing models...


In [6]:
accuracies

{'mlp': [0.2207, 0.2449, 0.2485]}

In [7]:
stats = {
    model_name: {
        "mean": float(np.mean(values)),
        "std": float(np.std(values))
    }
    for model_name, values in accuracies.items()
}
stats

{'mlp': {'mean': 0.23803333333333332, 'std': 0.012344319431310183}}